In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"thenik3","key":"2e8b6aab5408d942ff03d6881afb0729"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            109  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              215KB  2021-03-04 07:44:29             19  
nickuzmenkov/ranzcr-clip-kfold-tfrecords                    RANZCR CLiP KFold TFRecords                         2GB  2021-02-21 13:29:51              7  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44             54  
landrykezebou/lvzhdr-tone-mapping-benchmark-dataset-tmonet  LVZ-HDR Tone Map

In [ ]:
! kaggle datasets download -d apoorvwatsky/miraclvc1

100% 5.13G/5.13G [01:12<00:00, 99.0MB/s]
100% 5.13G/5.13G [01:12<00:00, 75.9MB/s]


In [ ]:
!ls

kaggle.json  miraclvc1.zip  sample_data


In [ ]:
! unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/dataset/M07/words/04/08/depth_013.png  
  inflating: dataset/dataset/M07/words/04/09/color_001.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_002.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_003.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_004.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_005.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_006.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_007.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_008.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_009.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_010.jpg  
  inflating: dataset/dataset/M07/words/04/09/color_011.jpg  
  inflating: dataset/dataset/M07/words/04/09/depth_001.png  
  inflating: dataset/dataset/M07/words/04/09/depth_002.png  
  inflating: dataset/dataset/M07/words/04/09/depth_003.png  
  inflating: dataset/dataset/M07/w

In [ ]:
! rm *.zip

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
import imutils
import dlib
import cv2
import imageio
from imutils import face_utils
from skimage.transform import resize

The below code is to detect lips from full photos in dataset, it is inspired from given link
[link text](https://www.pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/)

In [ ]:

def crop_and_save_image(img_path, write_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')
    # load the input image, resize it, and convert it to grayscale

    image = cv2.imread(img_path + '/' + img_name)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    if len(rects) > 1:
    	print( "ERROR: more than one face detected")
    	return
    if len(rects) < 1:
    	print( "ERROR: no faces detected")
    	return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        # clone = gray.copy()

        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))        
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)
        cv2.imwrite( write_path, roi)

In [ ]:
people = ['F01', 'F02', 'F04', 'F05', 'F06']
data_types = ['words']
folder_enum = ['01','02','03','04','05','06','07','08', '09', '10']
instances = ['01','02','03','04','05','06','07','08', '09', '10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web'] 
phrases = ['Stop navigation','Excuse me','I am sorry','Thank you','Good bye','I love this game','Nice to meet you','You are welcome','How are you?','Have a good time']
words_di = {i:words[i] for i in range(len(words))}
phrases_di = {i:phrases[i] for i in range(len(phrases))}

words_di
phrases_di

{0: 'Stop navigation',
 1: 'Excuse me',
 2: 'I am sorry',
 3: 'Thank you',
 4: 'Good bye',
 5: 'I love this game',
 6: 'Nice to meet you',
 7: 'You are welcome',
 8: 'How are you?',
 9: 'Have a good time'}

In [ ]:

for person_ID in people:
        if not os.path.exists('/content/cropped/cropped/' + person_ID ):
            os.mkdir('/content/cropped/cropped/' + person_ID + '/')

        for data_type in data_types:
            if not os.path.exists('/content/cropped/cropped/' + person_ID + '/' + data_type):
                os.mkdir('/content/cropped/cropped/' + person_ID + '/' + data_type)

            for phrase_ID in folder_enum:
                if not os.path.exists('/content/cropped/cropped/' + person_ID + '/' + data_type + '/' + phrase_ID):
                    # F01/phrases/01
                    os.mkdir('/content/cropped/cropped/' + person_ID + '/' + data_type + '/' + phrase_ID)

                for instance_ID in instances:
                    # F01/phrases/01/01
                    directory = '/content/dataset/dataset/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    dir_temp = '/content/cropped/cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
    #                 #print(directory)
                    filelist = os.listdir(directory)
                    if not os.path.exists('/content/cropped/cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID):
                        os.mkdir('/content/cropped/cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID)

                        for img_name in filelist:
                            if img_name.startswith('color'):
                                #image = imageio.imread(directory + '' + img_name)
                                crop_and_save_image(directory, dir_temp + '' + img_name, img_name)

In [ ]:
max_seq_length = 24

X_train_words = []
y_train_words = []
X_val_words = []
y_val_words = []
X_test_words = []
y_test_words = []


MAX_WIDTH = 100
MAX_HEIGHT = 100

people = ['F01', 'F02', 'F04', 'F05', 'F06']

In [ ]:

UNSEEN_VALIDATION_SPLIT = ['F05']
UNSEEN_TEST_SPLIT = ['F06']

directory = "/content/cropped/cropped"

for person_id in people:
    for data_type in data_types:
        for word_index, word in enumerate(folder_enum):
            for iteration in instances:
                path = os.path.join(directory, person_id, data_type, word, iteration)
                filelist = sorted(os.listdir(path + '/'))
                sequence = [] 
                for img_name in filelist:
                    if img_name.startswith('color'):
                        image = imageio.imread(path + '/' + img_name)
                        image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                        image = 255 * image
                        # Convert to integer data type pixels.
                        image = image.astype(np.uint8)
                        sequence.append(image)                        
                padding_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]                            
                sequence.extend(padding_array * (max_seq_length - len(sequence)))
                sequence = np.array(sequence)
                                
                if person_id in UNSEEN_TEST_SPLIT:
                    X_test_words.append(sequence)
                    y_test_words.append(word_index)
                elif person_id in UNSEEN_VALIDATION_SPLIT:
                    X_val_words.append(sequence)
                    y_val_words.append(word_index)
                else:
                    X_train_words.append(sequence)
                    y_train_words.append(word_index)    
    print(f'Finished reading images for person {person_id}.')    


Finished reading images for person F01.
Finished reading images for person F02.
Finished reading images for person F04.
Finished reading images for person F05.
Finished reading images for person F06.


In [ ]:
X_train_words = np.array(X_train_words)
X_val_words = np.array(X_val_words)
X_test_words = np.array(X_test_words)

print(X_train_words.shape)
print(X_val_words.shape)
print(X_test_words.shape)

(300, 24, 100, 100)
(100, 24, 100, 100)
(100, 24, 100, 100)


In [ ]:
y_train_words = np.array(y_train_words)
y_val_words = np.array(y_val_words)
y_test_words = np.array(y_test_words)

print(y_train_words.shape)
print(y_val_words.shape)
print(y_test_words.shape)

(300,)
(100,)
(100,)


In [ ]:
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    X = (X - v_min)/(v_max - v_min)
    X = np.nan_to_num(X)
    return X

In [ ]:
from keras.utils import np_utils

In [ ]:
X_train_words = normalize_it(X_train_words)
X_val_words = normalize_it(X_val_words)
X_test_words = normalize_it(X_test_words)

y_train_words = np_utils.to_categorical(y_train_words, 10)
y_test_words = np_utils.to_categorical(y_test_words, 10)
y_val_words = np_utils.to_categorical(y_val_words, 10)

from sklearn.utils import shuffle
X_train_words, y_train_words = shuffle(X_train_words, y_train_words, random_state=0)
X_test_words, y_test_words = shuffle(X_test_words, y_test_words, random_state=0)
X_val_words, y_val_words = shuffle(X_val_words, y_val_words, random_state=0)

In [ ]:
X_train_words = np.expand_dims(X_train_words, axis=4)
X_val_words = np.expand_dims(X_val_words, axis=4)
X_test_words = np.expand_dims(X_test_words, axis=4)

print(X_train_words.shape)
print(X_val_words.shape)
print(X_test_words.shape)

(300, 24, 100, 100, 1)
(100, 24, 100, 100, 1)
(100, 24, 100, 100, 1)


In [ ]:
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
from keras.utils import plot_model
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

In [ ]:
model = Sequential()
# 1st layer group
model.add(Conv3D(32, (3, 3, 3), strides = 1, input_shape=(24, 100, 100, 1), activation='relu', padding='valid'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(64, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(128, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add((Flatten()))

# # FC layers group
model.add(Dense(2048, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 22, 98, 98, 32)    896       
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 11, 49, 49, 32)    0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 9, 47, 47, 64)     55360     
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 4, 23, 23, 64)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 2, 21, 21, 128)    221312    
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 1, 10, 10, 128)    0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)            

In [ ]:
history = model.fit(X_train_words, y_train_words, validation_data=(X_val_words, y_val_words), epochs=45)

Epoch 1/45
10/10 [==============================] - 391s 39s/step - loss: 2.3524 - accuracy: 0.1148 - val_loss: 2.3034 - val_accuracy: 0.1000
Epoch 2/45
10/10 [==============================] - 389s 39s/step - loss: 2.3077 - accuracy: 0.0820 - val_loss: 2.3021 - val_accuracy: 0.1000
Epoch 3/45
10/10 [==============================] - 390s 39s/step - loss: 2.2984 - accuracy: 0.1252 - val_loss: 2.3045 - val_accuracy: 0.1000
Epoch 4/45
10/10 [==============================] - 392s 39s/step - loss: 2.2858 - accuracy: 0.1682 - val_loss: 2.2977 - val_accuracy: 0.1300
Epoch 5/45
10/10 [==============================] - 391s 39s/step - loss: 2.2321 - accuracy: 0.1496 - val_loss: 2.3043 - val_accuracy: 0.1000
Epoch 6/45
10/10 [==============================] - 390s 39s/step - loss: 2.0106 - accuracy: 0.3059 - val_loss: 2.4918 - val_accuracy: 0.1600
Epoch 7/45
10/10 [==============================] - 390s 39s/step - loss: 1.6778 - accuracy: 0.4353 - val_loss: 3.4856 - val_accuracy: 0.2200
Epoch 

KeyboardInterrupt: ignored

In [ ]:
ypred = model.predict(X_test_words)

In [ ]:
predicted_words = [words[i] for i in np.argmax(ypred, axis=1)]
actual_words = [words[i] for i in np.argmax(y_test_words, axis=1)] 

In [ ]:
correct = 0
for p, a in zip(predicted_words, actual_words):
    if p == a:
        correct += 1

accuracy = correct/len(actual_words)
print(f"Accuracy = {accuracy} on completely unseen data")

Accuracy = 0.41 on completely unseen data
